In [8]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import keras
from scipy.sparse import csr_matrix
from scipy.sparse import hstack

import gc
import re
import math
from time import time
from collections import Counter
import tensorflow as tf
import csv


from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.models import Model,load_model

from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Input
from sklearn.metrics import mean_squared_log_error

In [9]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

import plotly 
import plotly.offline as offline
import plotly.graph_objs as go
from collections import Counter

import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random
#from wordcloud import WordCloud
import nltk
from tqdm import tqdm
warnings.filterwarnings("ignore")

#from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [10]:
data=pd.read_csv('train.tsv', sep='\t')
x_test=pd.read_csv('test_stg2.tsv', sep='\t')
data.head()


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [11]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

# Combining all the above stundents 
from tqdm import tqdm
def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [12]:
def prediction(input_, model):
    """this function predicts the price basing on the trained model"""
    batchsize = 256
    preds = model.predict(input_.tocsr(), batch_size=batchsize)
    preds = np.exp(preds)+1
    return preds

In [54]:
import math
def handle_data(data):
    def transform_category_name(category_name):
        try:
            main, sub1, sub2= category_name.split('/')
            return main, sub1, sub2
        except:
            return ('missing', 'missing', 'missing')
        
    def branded(brand_name):
        is_branded=[]
        for i in data['brand_name']:
            if i=='missing': 
                is_branded.append(0) 
            else: 
                is_branded.append(1)
            return is_branded

    if type(data)==type(pd.DataFrame()):   #checking if it is a dataframe or not
        data['category_name'].fillna(value='missing',inplace=True)
        
        data['category_main'], data['category_sub1'], data['category_sub2'] = zip(*data['category_name'].apply(transform_category_name))
        data['brand_name'].fillna(value='missing',inplace=True)
        data['is_branded']=zip(*data['brand_name'].apply(branded))
        data['item_description'].fillna(value='No description yet',inplace=True)
        data['name']= data['name']
        data['shipping']=data['shipping']
        data['item_condition_id']=data['item_condition_id']
        
        if 'price' in data.columns:
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id','price']]
        else:
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id']]
        
    else:      #if the given data is a data point
        if type(data['category_name'])==type(float()):  #checking for nan values in category_name
            data['category_name'].fillna("missing",inplace=True)
            data['category_main'], data['category_sub1'], data['category_sub2'] = zip(*data['category_name'].apply(transform_category_name))

        if type(data['brand_name'])==type(float()):     #checking for nan values in brand_name
            data['brand_name'].fillna(value='missing',inplace=True)
            data['is_branded']=zip(*data['brand_name'].apply(branded))

        if type(data['item_description'])==type(float()):  #checking for nan values in item_description
            data['item_description'].fillna("No description yet",inplace=True)
            #data['category_main'], data['category_sub1'], data['category_sub2'] =data['category_name'].split('/')
        if data['brand_name']=='missing': 
            data["is_branded"]=0
        else: 
            data["is_branded"]=1
        data['name']= data['name']
        data['shipping']=data['shipping']
        data['item_condition_id']=data['item_condition_id']

        if 'price' in dict(data).keys():  #if price exits we will return it
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id','price']]
        else:
            return data[['name','item_description','brand_name','category_main','category_sub1','category_sub2','is_branded','shipping','item_condition_id']]
    return False

In [55]:
def price_suggestion(X):

    X=handle_data(X)          
    if 'price' in dict(X).keys():
        target=X['price']  #checking if price contains in it or not
    else:
        target='we predicted it'
    vectorizer_name=pickle.load(open("vectorizer_name.pkl","rb"))
    bow_name=vectorizer_name.transform(X['name'].values)  #name vectorization
    
    vectorizer_item=pickle.load(open("vectorizer_item.pkl", 'rb'))
    bow_item=vectorizer_item.transform(X['item_description'].values) 

    vectorizer_cat=pickle.load(open("vectorizer_cat.pkl","rb"))
    bow_cat=vectorizer_cat.transform(X['category_main'].values) 
    
    vectorizer_sub1=pickle.load(open("vectorizer_sub1.pkl","rb"))
    bow_sub1=vectorizer_sub1.transform(X['category_sub1'].values)
    #text vectorization
    
    vectorizer_sub2=pickle.load(open("vectorizer_sub2.pkl","rb"))
    bow_sub2=vectorizer_sub2.transform(X['category_sub2'].values)
    #text vectorization
    
    vectorizer_brand=pickle.load(open("vectorizer_brand.pkl","rb"))
    bow_brand=vectorizer_brand.transform(X['brand_name'].values)  #text vectorization

    is_brand_oe=pickle.load(open("is_branded_oe.pkl","rb"))
    is_brand=is_brand_oe.transform(X[['is_branded']])
    
    item_cond_oe=pickle.load(open("item_cond.pkl","rb"))
    item_cond=item_cond_oe.transform(X[['item_condition_id']])
    
    shipping_oe=pickle.load(open("shipping_oe.pkl","rb"))
    shipping=shipping_oe.transform(X[['shipping']])
    


    concat=hstack([bow_name,bow_item,bow_cat,bow_sub1,bow_sub2,bow_brand,is_brand,item_cond,shipping])   #concatinating all the features
    best_model = load_model('baseline_mlp_model_epochs:001-val_loss:0.183.hdf5')

    #preds = best_model.predict(X_te.tocsr())[:, 0]
    predicted_price=prediction(concat,best_model)
    return predicted_price[0],target  

In [56]:
predicted,target=price_suggestion(x_test.reindex([110]))  #some random train data point.
if target!='we predicted it':
  print("Predicted price is: {} and Actual price of the product is: {}".format(predicted,target))
else:
  print("Predicted price for the given product is: {}".format(predicted[0]))

Predicted price for the given product is: 5.254732131958008


In [57]:
from sklearn.metrics import mean_squared_error
def error_metric(X,Y):
  """this function returns the rmsle error on the target and predicted score
     Input_format: X: data point(must be 7 or 8 dimensional datapoint) (vector)
                   Y: float 
     Output format: float (rmsle score) """
  predicted_score=price_suggestion(X)
  return rmsle([Y],[predicted_score[0]])

In [63]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [66]:
error=error_metric(data.reindex([10]),data.reindex([10])['price'])  #some random train data point
print("RMSLE on given datapoint is: ",error)

error=error_metric(data.reindex([170]),data.reindex([170])['price'])  #some random train data point
print("RMSLE on given datapoint is: ",error)

RMSLE on given datapoint is:  1.3077167567860957
RMSLE on given datapoint is:  0.8080904444622989
